In [17]:
import subprocess
from tika import parser #pip install tika REQUIERE JDK
import unidecode #pip install unidecode

In [23]:

raw = parser.from_file('data/planes_materias/pdf/1.pdf')
#print(raw['content'])
lines = raw['content']

lines = unidecode.unidecode(lines[77:])
print(lines)



f = open('data/planes_materias/txt/1.txt', 'w')
f.write(lines)
f.close()

#open and read the file after the appending:
#f = open("demofile2.txt", "r")
#print(f.read())




FACULTAD DE CIENCIAS

CARRERA DE MATEMATICO

ALGEBRA MODERNA I

SEMESTRE: QUINTO
CLAVE: 0001

HORAS A LA SEMANA/SEMESTRE

TEORICAS PRACTICAS CREDITOS
5/80 0 10

CARACTER: OBLIGATORIA.
MODALIDAD: CURSO.
SERIACION INDICATIVA ANTECEDENTE: Algebra Lineal II, Calculo Diferencial
e Integral III.
SERIACION INDICATIVA SUBSECUENTE: Algebra Moderna II, Analisis Matemati-
co III, Historia de las Matematicas II, Seminario de Algebra (A o B), Seminario
de Analisis Combinatorio, Seminario de Analisis Matematico (A o B), Seminario
de Filosofia de las Matematicas, Seminario de Geometria (A o B), Seminario de
Historia y Filosofia de la Ciencia I, Seminario de Topologia (A o B), Seminario
sobre la Ensenanza de las Matematicas I, Topologia II, Variable Compleja II.

OBJETIVO(S): Introducir al alumno a la teoria general de grupos.

NUM. HORAS UNIDADES TEMATICAS

25 1. Teoria elemental de grupos
1.1 Grupos.
1.2 Subgrupos.
1.3 Grupos ciclicos.
1.4 Grupos de permutaciones. Ciclos, transposiciones, paridad y 